第五章：这章讲的slam 的相机成像相关的知识

并且讲了Opencv的基础操作

相机的内参是出厂就固定了，外参数就是上一章节的位置与旋转。

又说明了单目相机，双目相机，深度相机等基本原理

undistortImage.cpp
去畸变的这个只有书上有，这边 基本操作就行了



In [1]:
import cv2
import numpy as np
import time

def main():
    # 读取命令行参数指定的图像
    import sys
    if len(sys.argv) < 2:
        print("请提供图像文件路径作为参数")
        return
    
    image_path = sys.argv[1]
    image = cv2.imread(image_path)

    # 判断图像文件是否正确读取
    if image is None:
        print(f"文件 {image_path} 不存在.")
        return

    # 输出基本信息
    print(f"图像宽为 {image.shape[1]}, 高为 {image.shape[0]}, 通道数为 {image.shape[2]}")
    cv2.imshow("image", image)
    cv2.waitKey(0)

    # 判断image的类型
    if image.dtype != np.uint8 or (image.ndim != 2 and image.ndim != 3):
        print("请输入一张彩色图或灰度图.")
        return

    # 遍历图像
    start_time = time.time()
    for y in range(image.shape[0]):
        for x in range(image.shape[1]):
            pixel = image[y, x]
            # 这里可以对像素进行操作，比如:
            # for c in range(image.shape[2]):
            #     data = pixel[c]

    end_time = time.time()
    print(f"遍历图像用时：{end_time - start_time:.6f} 秒。")

    # 关于图像的拷贝
    image_another = image.copy()
    image_another[0:100, 0:100] = 0  # 将左上角100*100的块置零
    cv2.imshow("image", image)
    cv2.waitKey(0)

    # 使用copy()函数来拷贝数据
    image_clone = image.copy()
    image_clone[0:100, 0:100] = 255
    cv2.imshow("image", image)
    cv2.imshow("image_clone", image_clone)
    cv2.waitKey(0)

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


文件 --f=/Users/bytedance/Library/Jupyter/runtime/kernel-v367e4a90032ee809008e86b4ff33e200e136fafca.json 不存在.


[ WARN:0@0.008] global loadsave.cpp:241 findDecoder imread_('--f=/Users/bytedance/Library/Jupyter/runtime/kernel-v367e4a90032ee809008e86b4ff33e200e136fafca.json'): can't open/read file: check file path/integrity


作者 书上有两个脚本这里没有额， 只有这边从joinMap文件夹中抽了几张照片复制到这边尝试复现一下双目slam的建图

In [ ]:
import cv2
import numpy as np
from scipy.spatial.transform import Rotation as R
import open3d as o3d
import matplotlib.pyplot as plt 

def main():
    # 内参
    fx, fy, cx, cy = 718.856, 718.856, 607.1928, 185.2157
    # 基线
    b = 0.573

    # 读取图像
    left = cv2.imread('4.png', 0)  # 请替换为实际的左图文件名
    right = cv2.imread('5.png', 0)  # 请替换为实际的右图文件名

    # 创建SGBM对象
    sgbm = cv2.StereoSGBM_create(
        minDisparity=0,
        numDisparities=96,
        blockSize=9,
        P1=8 * 9 * 9,
        P2=32 * 9 * 9,
        disp12MaxDiff=1,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32
    )

    # 计算视差
    disparity_sgbm = sgbm.compute(left, right)
    disparity = disparity_sgbm.astype(np.float32) / 16.0

    # 生成点云
    pointcloud = []

    # 如果你的机器慢，可以将步长改为2
    for v in range(0, left.shape[0], 1):  # 如果需要，改为 range(0, left.shape[0], 2)
        for u in range(0, left.shape[1], 1):  # 如果需要，改为 range(0, left.shape[1], 2)
            if disparity[v, u] <= 10.0 or disparity[v, u] >= 96.0:
                continue

            # 前三维为xyz，第四维为颜色
            point = [0, 0, 0, left[v, u] / 255.0]

            # 根据双目模型计算point的位置
            x = (u - cx) / fx
            y = (v - cy) / fy
            depth = fx * b / disparity[v, u]
            point[0] = x * depth
            point[1] = y * depth
            point[2] = depth

            pointcloud.append(point)
    #画出深度图
    print("深度")

    cv2.imshow("disparity", disparity / 96.0)
    cv2.waitKey(0)

    # 画出点云
    print("画出点云")
    show_point_cloud(pointcloud)

# def show_point_cloud(pointcloud):
#     # 将点云数据转换为Open3D格式
#     pcd = o3d.geometry.PointCloud()
#     points = np.array(pointcloud)[:, :3]  # 取前三列作为坐标
#     colors = np.array(pointcloud)[:, 3:]  # 取最后一列作为颜色
#     colors = np.repeat(colors, 3, axis=1)  # 将灰度值重复三次得到RGB
#     pcd.points = o3d.utility.Vector3dVector(points)
#     pcd.colors = o3d.utility.Vector3dVector(colors)

#     # 可视化点云
#     o3d.visualization.draw_geometries([pcd])
def show_point_cloud(pointcloud):
    # 将点云数据转换为Open3D格式
    pcd = o3d.geometry.PointCloud()
    points = np.array(pointcloud)[:, :3]  # 取前三列作为坐标
    colors = np.array(pointcloud)[:, 3:]  # 取最后一列作为颜色
    colors = np.repeat(colors, 3, axis=1)  # 将灰度值重复三次得到RGB
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)

    # 创建可视化器
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)

    # 更新渲染
    vis.update_geometry(pcd)
    vis.poll_events()
    vis.update_renderer()

    # 捕获屏幕图像
    image = vis.capture_screen_float_buffer()
    
    # 保存图像（可选）
    # plt.imsave("pointcloud.png", np.asarray(image))

    # 显示图像
    plt.imshow(np.asarray(image))
    plt.show()

    # 关闭可视化器
    vis.destroy_window()


if __name__ == "__main__":
    main()


/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Matplotlib is building the font cache; this may take a moment.


深度


2025-02-11 12:54:17.299 Python[1532:712437] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-11 12:54:17.299 Python[1532:712437] +[IMKInputSession subclass]: chose IMKInputSession_Modern
